In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cs_training = pd.read_csv(r"E:\牛奕彤文件\PHBS课程\MODULE3\Machine Learning for Finance\cs-training.csv\cs-training.csv")
del cs_training['Unnamed: 0']

In [4]:
cs_training = cs_training.fillna(cs_training.mean())
cs_training

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.000000,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.000000,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.000000,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.000000,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.000000,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.040674,74,0,0.225131,2100.000000,4,0,1,0,0.0
149996,0,0.299745,44,0,0.716562,5584.000000,4,0,1,0,2.0
149997,0,0.246044,58,0,3870.000000,6670.221237,18,0,1,0,0.0
149998,0,0.000000,30,0,0.000000,5716.000000,4,0,0,0,0.0


## Divide data into train and test set

In [5]:
from sklearn.model_selection import train_test_split

X, y = cs_training.iloc[:, 1:].values, cs_training.iloc[:, 0].values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, 
                     stratify=y,
                     random_state=0)

## Use GridSearch to fine-tune hyperparameters, Implement 5-fold CV

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1, solver='lbfgs'))

# param_range_1 = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_range_1 = np.logspace(10 ** (-7), 1000)
param_range_2 = [3, 4, 5]

param_grid = {'logisticregression__C': param_range, 'pca__n_components': param_range_2}

gs = GridSearchCV(estimator=pipe_lr,
                 param_grid = param_grid,
                  cv=5,
                 scoring='accuracy', 
                  refit=True,
                  n_jobs=-1)

gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

C:\Users\NYT\AppData\Roaming\Python\Python39\site-packages\numpy\core\function_base.py:298: RuntimeWarning: overflow encountered in power
  return _nx.power(base, y)


0.9333523809523809
{'logisticregression__C': 0.0001, 'pca__n_components': 3}


In [18]:
clf = gs.best_estimator_

In [19]:
clf

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=3)),
                ('logisticregression',
                 LogisticRegression(C=0.0001, random_state=1))])

In [20]:
print('Test accuracy: %.5f' % clf.score(X_test, y_test))

Test accuracy: 0.93327
